<a href="https://colab.research.google.com/github/perpendicooler/Fake-News-Prediciton/blob/main/Fake_news_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **About the Dataset:**



> 1.Id : Unique id for a news article\
  2.Tittle : The tittle of a news article\
  3.Author : Author of the news article\
  4.Text : The text of the article could be incomplete\
  5.Label : A labelthath makes whether the news article is real or fake\




```
      1: fake news
      0: real news
```



Importing the Dependencies


In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score



Downloading the nltk stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Printing those stopwrods


In [ ]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

**Data Pre-processing**

In [53]:
news_dataset = pd.read_csv("/content/train.csv",encoding =('ISO-8859-1'),low_memory =False)

In [ ]:
news_dataset.shape


In [76]:
# print the first 5 rows of the dataframe
news_dataset.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...,1,Darrell Lucus House Dem Aide: We Didnât Even...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [77]:
# counting the number of missing values in the dataset
news_dataset.isnull().sum()

id         0
title      0
author     0
text       0
label      0
content    0
dtype: int64

In [78]:
# replacing the null values with empty string
news_dataset = news_dataset.fillna('')

In [79]:
# merging the author name and news title
news_dataset['content'] = news_dataset['author']+' '+news_dataset['title']

In [94]:
print(news_dataset['content'])

0        darrel lucu hou dem aid even see comey letter ...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20982    jerom hudson rapper trump poster child white s...
20983    benjamin hoffman n f l playoff schedul matchup...
20984    michael j de la merc rachel abram maci said re...
20985    alex ansari nato russia hold parallel exerci b...
20986                            david swanson keep f aliv
Name: content, Length: 20987, dtype: object


In [101]:
# separating the data & label
X = news_dataset.drop(columns='label', axis=1)
Y = news_dataset['label']

In [100]:
print(X)
print(Y)


          id                                              title  \
0          0  House Dem Aide: We Didnât Even See Comeyâs...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20982  20795  Rapper T.I.: Trump a âPoster Child For White...   
20983  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20984  20797  Macyâs Is Said to Receive Takeover Approach ...   
20985  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20986  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

Stemming:

Stemming is the process of reducing a word to its Root word

example: actor, actress, acting --> act

In [102]:
port_stem = PorterStemmer()

In [103]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [104]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [105]:
print(news_dataset['content'])

0        darrel lucu hou dem aid even see comey letter ...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20982    jerom hudson rapper trump poster child white s...
20983    benjamin hoffman n f l playoff schedul matchup...
20984    michael j de la merc rachel abram maci said re...
20985    alex ansari nato russia hold parallel exerci b...
20986                            david swanson keep f aliv
Name: content, Length: 20987, dtype: object


In [127]:
#separating the data and label
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [128]:
print(X)

['darrel lucu hou dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exerci balkan'
 'david swanson keep f aliv']


In [129]:
print(Y)

[1 0 1 ... 0 1 1]


In [130]:
Y.shape

(20987,)

In [133]:
#converting the texual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X=vectorizer.transform(X)

In [134]:
print(X)

  (0, 15545)	0.2850318003826496
  (0, 13342)	0.25679061533080033
  (0, 8818)	0.363722423466063
  (0, 8542)	0.2923012134813304
  (0, 7608)	0.24780539728251844
  (0, 6928)	0.21833161341076707
  (0, 4916)	0.23335364573782766
  (0, 3759)	0.2702927390838912
  (0, 3567)	0.3600225533782931
  (0, 2933)	0.24655833776469496
  (0, 2464)	0.36777517260667286
  (0, 263)	0.26986538190046394
  (1, 16646)	0.30213217442526635
  (1, 6742)	0.19192928976690793
  (1, 5439)	0.7178620551937107
  (1, 3535)	0.2654607362304112
  (1, 2790)	0.19228907689014405
  (1, 2206)	0.36990210771041004
  (1, 1877)	0.1562307868727455
  (1, 1481)	0.2964342203473744
  (2, 15470)	0.4158120561509754
  (2, 9528)	0.49383232853982384
  (2, 5901)	0.34744900064136963
  (2, 5325)	0.38556160215913693
  (2, 3072)	0.46131106499979796
  :	:
  (20984, 12992)	0.24873757218136416
  (20984, 12222)	0.2722429625598303
  (20984, 12017)	0.24826723759815406
  (20984, 10212)	0.08047672915313847
  (20984, 9496)	0.17459226351668378
  (20984, 9427)	0.2

Splitting the dataset to training & test data

In [177]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.1, stratify=Y, random_state=1)

Training the Model: Logistic Regression

In [178]:
model = LogisticRegression()

In [179]:
model.fit(X_train, Y_train)

LogisticRegression()

Evaluation


accuracy score

In [180]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [181]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9870817450232953


In [182]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [183]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.9785612196283945


Making a Predictive System

In [189]:
X_new = X_test[1000]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[0]
The news is Real


In [190]:
print(Y_test[1000])

0
